In [1]:
#Auotograd
import torch

In [2]:
x=torch.ones(2,2,requires_grad=True)
print(x)


tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
y=x+2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [4]:
#y是一次运算的结果，所以是有grad_fn的
print(y.grad_fn)

In [5]:
z=y*y*3
out=z.mean()
print(z,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [6]:
a=torch.rand(2,2)
a=((a*3)/(a-1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b=(a*a).sum()
print(b.grad_fn)


False
True


In [7]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [8]:
x=torch.randn(3,requires_grad=True)
y=x*2
while y.data.norm()<1000:
    y=y*2
print(y)

tensor([ 1503.1361,  -423.3788, -1084.8182], grad_fn=<MulBackward0>)


In [10]:
v=torch.tensor([0.1,1.0,0.0001],dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [1]:
#卷积，填充
import torch
from torch import nn
#计算卷积层函数，对输入和输出进行升维，降维
def comp_conv2d(conv2d,X):
    X=X.view((1,1)+X.shape)#(1,1)代表批量大小和通道数
    Y=conv2d(X)
    return Y.view(Y.shape[2:])#不关心前两维，批量与通道
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3,padding=1)#注意这个是在两侧分别填充一行/列，所以总共是填充了2，填出10x10
X=torch.rand(8,8)
comp_conv2d(conv2d,X).shape#10 10用3 3卷积，得到8 8

torch.Size([8, 8])

In [2]:
#当然宽和高不同也可以设置不同的填充数使输入输出有相同的宽和高
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=(5,3),padding=(2,1))#高为5宽为3的卷积核，填充是2，1
comp_conv2d(conv2d,X).shape


torch.Size([8, 8])

In [6]:
#设置步幅
conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1,stride=2)
comp_conv2d(conv2d,X).shape

torch.Size([4, 4])

In [8]:
#尝试池化层
X=torch.arange(16,dtype=torch.float).view((1,1,4,4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [9]:
pool2d=nn.MaxPool2d(3)
pool2d(X)#默认步幅形状是(3,3)

tensor([[[[10.]]]])

In [11]:
pool2d=nn.MaxPool2d(3,padding=1,stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [ ]:
#define the network
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()
        #一个输入通道，6个输出通道，3x3卷积核
        #kernel1
        self.conv1=nn.Conv2d(1,6,3)#对应的是输入，输出通道，核大小
        self.conv2=nn.Conv2d(6,16,3)
        #affine计算
        self.fc1=nn.Linear(16*6*6,120)
        self.fc2=nn.Linear(120,84)